In [1]:
# === CELDA 1: Imports y rutas ===
import os
import pandas as pd
import numpy as np
from datetime import datetime

# Rutas base (ajusta si cambiaste)
BASE_DIR = r"C:\Users\User\Downloads\Mini reinanse"
DATA_DIR = os.path.join(BASE_DIR, "data")
PROCESSED_DIR = os.path.join(DATA_DIR, "processed")
ESTADO_DIR = os.path.join(DATA_DIR, "estado")
OUTPUT_DIR = os.path.join(DATA_DIR, "processed")
os.makedirs(OUTPUT_DIR, exist_ok=True)

print("✅ Rutas OK")


✅ Rutas OK


In [2]:
# === CELDA 2: Cargar dataset maestro ===
file_parquet = os.path.join(PROCESSED_DIR, "dataset_master.parquet")
file_csv = os.path.join(PROCESSED_DIR, "dataset_master.csv")

if os.path.exists(file_parquet):
    df = pd.read_parquet(file_parquet)
    print("✅ Cargado desde Parquet")
elif os.path.exists(file_csv):
    df = pd.read_csv(file_csv)
    print("✅ Cargado desde CSV")
else:
    raise FileNotFoundError("❌ No encontré dataset_master.parquet ni dataset_master.csv")

# Tipos y orden
df["time"] = pd.to_datetime(df["time"], errors="coerce")
df = df.sort_values(["symbol", "time"]).reset_index(drop=True)

# Validaciones mínimas
req_cols = ["symbol", "time", "return", "drawdown", "mom_63", "vol_63"]
faltantes = [c for c in req_cols if c not in df.columns]
if faltantes:
    raise ValueError(f"❌ Faltan columnas en dataset_master: {faltantes}")

print("✅ Filas:", df.shape[0], "| Columnas:", df.shape[1], "| Símbolos:", df["symbol"].nunique())
df.head()


✅ Cargado desde Parquet
✅ Filas: 509661 | Columnas: 16 | Símbolos: 681


,time,retornos,equity,symbol,return,log_return,mom_63,drawdown,cumulative_return,MA_20,MA_50,MA_200,mom_21,vol_20,vol_63,rolling_peak
0,2022-11-18,-0.010611,146.07,A.US,-0.010611,-0.010668,-0.027932,0.000000,-0.010611,153.0775,152.6428,135.446,0.012802,0.025984,0.019476,146.07
1,2022-11-21,-0.010611,144.52,A.US,-0.010611,-0.010668,-0.027932,-0.010611,-0.010611,153.0775,152.6428,135.446,0.012802,0.025984,0.019476,146.07
2,2022-11-22,0.085940,156.94,A.US,0.085940,0.082446,-0.027932,0.000000,0.075328,153.0775,152.6428,135.446,0.012802,0.025984,0.019476,156.94
3,2022-11-23,-0.010768,155.25,A.US,-0.010768,-0.010827,-0.027932,-0.010768,0.064560,153.0775,152.6428,135.446,0.012802,0.025984,0.019476,156.94
4,2022-11-25,0.009275,156.69,A.US,0.009275,0.009233,-0.027932,-0.001593,0.073835,153.0775,152.6428,135.446,0.012802,0.025984,0.019476,156.94


In [3]:
# === CELDA 3: Métricas rolling y anualización ===
# Rolling mean/stdev 63d (≈ 3 meses bursátiles)
df["mean_ret_63"] = df.groupby("symbol")["return"].transform(lambda x: x.rolling(63, min_periods=30).mean())
df["std_ret_63"]  = df.groupby("symbol")["return"].transform(lambda x: x.rolling(63, min_periods=30).std())

# Sharpe 63d anualizado: sqrt(252) * mean / std
df["sharpe_63"] = np.where(
    df["std_ret_63"] > 0,
    (df["mean_ret_63"] * np.sqrt(252)) / df["std_ret_63"],
    np.nan
)

# Momento ya existe como mom_63 (de 01B)
# Volatilidad ya existe como vol_63 (std diaria 63d)

# Drawdown ya está (instantáneo). Calculamos max_drawdown total por símbolo (para info):
def max_drawdown_from_drawdown(series):
    # serie de drawdown (negativa o 0) → min es el máximo drawdown
    return series.min()

max_dd = df.groupby("symbol")["drawdown"].agg(max_drawdown_from_drawdown).rename("max_drawdown_total")
df = df.merge(max_dd, on="symbol", how="left")

print("✅ Métricas rolling calculadas")
df[["symbol","time","return","mom_63","sharpe_63","drawdown"]].head()


✅ Métricas rolling calculadas


,symbol,time,return,mom_63,sharpe_63,drawdown
0,A.US,2022-11-18,-0.010611,-0.027932,NaN,0.000000
1,A.US,2022-11-21,-0.010611,-0.027932,NaN,-0.010611
2,A.US,2022-11-22,0.085940,-0.027932,NaN,0.000000
3,A.US,2022-11-23,-0.010768,-0.027932,NaN,-0.010768
4,A.US,2022-11-25,0.009275,-0.027932,NaN,-0.001593


In [4]:
# === CELDA 4: Cargar estados HMM clasificados y unir ===
import glob

files_estados = glob.glob(os.path.join(ESTADO_DIR, "estado_*_clasificado.csv"))
if not files_estados:
    raise FileNotFoundError("❌ No encontré archivos estado_*_clasificado.csv en data/estado")

dfs_estado = []
for f in files_estados:
    tmp = pd.read_csv(f)
    # Normalizar columnas esperadas
    # Debe contener: time, symbol, estado, estado_clasificado
    # Asegurar time como datetime
    if "time" not in tmp.columns:
        continue
    tmp["time"] = pd.to_datetime(tmp["time"], errors="coerce")
    # Si falta 'symbol', inferirlo del nombre de archivo
    if "symbol" not in tmp.columns:
        # estado_<SYMBOL>_clasificado.csv
        base = os.path.basename(f)
        sym = base.replace("estado_","").replace("_clasificado.csv","")
        tmp["symbol"] = sym
    # Quedarnos con columnas clave
    keep = [c for c in ["time","symbol","estado","estado_clasificado"] if c in tmp.columns]
    tmp = tmp[keep].dropna(subset=["time","symbol"])
    dfs_estado.append(tmp)

df_estados = pd.concat(dfs_estado, ignore_index=True).drop_duplicates(subset=["symbol","time"])

# Merge por símbolo+fecha
df = df.merge(df_estados, on=["symbol","time"], how="left")

# Asegurar valores válidos en estado_clasificado
df["estado_clasificado"] = df["estado_clasificado"].fillna("SIN_ESTADO")

print("✅ Estados HMM unidos al dataset")
df[["symbol","time","estado_clasificado"]].head()


✅ Estados HMM unidos al dataset


,symbol,time,estado_clasificado
0,A.US,2022-11-18,LATERAL
1,A.US,2022-11-21,LATERAL
2,A.US,2022-11-22,LATERAL
3,A.US,2022-11-23,LATERAL
4,A.US,2022-11-25,LATERAL


In [5]:
# === CELDA 5: Score compuesto con percentiles diarios y NO GO por HMM ===

def pct_rank(s):
    return s.rank(pct=True)

# Percentiles cruzados por fecha (cross-sectional)
df["pr_mom63"]    = df.groupby("time")["mom_63"].transform(pct_rank)
df["pr_sharpe63"] = df.groupby("time")["sharpe_63"].transform(pct_rank)

# Score final [0..1]
df["score"] = 0.5*df["pr_mom63"] + 0.5*df["pr_sharpe63"]

# Clasificación base por score y métricas (criterios que usabas)
def clasificacion_base(row):
    # Umbrales típicos de tu sistema
    if pd.isna(row["score"]) or pd.isna(row["sharpe_63"]) or pd.isna(row["drawdown"]):
        return "Incompleto"
    if row["score"] >= 0.15 and row["sharpe_63"] > 0.5 and row["drawdown"] > -0.20:
        return "Compra Fuerte"
    if 0.08 <= row["score"] < 0.15:
        return "Neutro"
    return "Venta"

df["clasificacion_base"] = df.apply(clasificacion_base, axis=1)

# Regla NO GO: si HMM dice BAJISTA ⇒ Venta/NO GO
def aplicar_no_go(row):
    if row["estado_clasificado"] == "BAJISTA":
        return "Venta (NO GO HMM)"
    return row["clasificacion_base"]

df["clasificacion"] = df.apply(aplicar_no_go, axis=1)

print("✅ Score y clasificación generados")
df[["symbol","time","score","sharpe_63","drawdown","estado_clasificado","clasificacion"]].head()


✅ Score y clasificación generados


,symbol,time,score,sharpe_63,drawdown,estado_clasificado,clasificacion
0,A.US,2022-11-18,NaN,NaN,0.000000,LATERAL,Incompleto
1,A.US,2022-11-21,NaN,NaN,-0.010611,LATERAL,Incompleto
2,A.US,2022-11-22,NaN,NaN,0.000000,LATERAL,Incompleto
3,A.US,2022-11-23,NaN,NaN,-0.010768,LATERAL,Incompleto
4,A.US,2022-11-25,NaN,NaN,-0.001593,LATERAL,Incompleto


In [6]:
# === CELDA 6: Snapshot de señales en la última fecha disponible ===

# Última fecha global del dataset
last_date = df["time"].max()

# Último registro por símbolo en esa fecha (si falta, tomamos el último disponible por símbolo)
df_last_day = df[df["time"] == last_date].copy()

# Si hay símbolos sin dato justo en last_date, completamos con el último disponible por símbolo
if df_last_day["symbol"].nunique() < df["symbol"].nunique():
    tail_by_sym = df.sort_values("time").groupby("symbol").tail(1)
    df_last_day = pd.concat([df_last_day, tail_by_sym[~tail_by_sym["symbol"].isin(df_last_day["symbol"])]]).drop_duplicates("symbol", keep="first")

# Campos finales
cols_finales = [
    "symbol","time","score","sharpe_63","drawdown","max_drawdown_total",
    "estado_clasificado","clasificacion"
]
df_signals = df_last_day[cols_finales].copy()

df_signals = df_signals.sort_values(["clasificacion","score","sharpe_63"], ascending=[True,False,False])
df_signals["fecha_generacion"] = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

# Export
# Guardamos como archivo previo al Notebook 04
out_file = os.path.join(OUTPUT_DIR, "score_previo_04.csv")
df_signals.to_csv(out_file, index=False, encoding="utf-8-sig")

print("✅ Archivo generado:", out_file)


print("✅ Señales del día generadas")
print("📁", out_file)
df_signals.head(20)


✅ Archivo generado: C:\Users\User\Downloads\Mini reinanse\data\processed\score_previo_04.csv
✅ Señales del día generadas
📁 C:\Users\User\Downloads\Mini reinanse\data\processed\score_previo_04.csv


,symbol,time,score,sharpe_63,drawdown,max_drawdown_total,estado_clasificado,clasificacion,fecha_generacion
484913,WDC.US,2025-11-14,0.998516,5.295517,-0.096183,-0.619636,SIN_ESTADO,Compra Fuerte,2025-11-17 16:03:02
38934,ARWR.US,2025-11-14,0.998516,5.111676,-0.027346,-0.759709,LATERAL,Compra Fuerte,2025-11-17 16:03:02
319284,MU.US,2025-11-14,0.997033,5.189222,-0.027368,-0.578056,SIN_ESTADO,Compra Fuerte,2025-11-17 16:03:02
94433,CIEN.US,2025-11-14,0.997033,4.931309,-0.089046,-0.455433,ALCISTA,Compra Fuerte,2025-11-17 16:03:02
270532,LITE.US,2025-11-14,0.989614,3.828602,-0.104829,-0.506728,ALCISTA,Compra Fuerte,2025-11-17 16:03:02
272032,LLY.US,2025-11-14,0.985905,4.918967,0.000000,-0.347938,ALCISTA,Compra Fuerte,2025-11-17 16:03:02
278034,LRCX.US,2025-11-14,0.980712,3.655366,-0.110704,-0.475018,SIN_ESTADO,Compra Fuerte,2025-11-17 16:03:02
284784,M.US,2025-11-14,0.977003,3.282542,-0.183565,-0.592805,SIN_ESTADO,Compra Fuerte,2025-11-17 16:03:02
203032,GOOG.US,2025-11-14,0.976261,4.140705,-0.050652,-0.296433,ALCISTA,Compra Fuerte,2025-11-17 16:03:02
203782,GOOGL.US,2025-11-14,0.976261,4.106767,-0.051033,-0.300891,ALCISTA,Compra Fuerte,2025-11-17 16:03:02


In [7]:
# === CELDA 7: Resumen de señales ===

top_compra = df_signals[df_signals["clasificacion"] == "Compra Fuerte"].sort_values("score", ascending=False).head(15)
conteo = df_signals["clasificacion"].value_counts(dropna=False)

print("📊 Conteo por clase:")
print(conteo)
print("\n🏆 TOP 15 Compra Fuerte:")
display(top_compra)


📊 Conteo por clase:
clasificacion
Venta                324
Compra Fuerte        191
Venta (NO GO HMM)    128
Neutro                38
Name: count, dtype: int64

🏆 TOP 15 Compra Fuerte:


,symbol,time,score,sharpe_63,drawdown,max_drawdown_total,estado_clasificado,clasificacion,fecha_generacion
484913,WDC.US,2025-11-14,0.998516,5.295517,-0.096183,-0.619636,SIN_ESTADO,Compra Fuerte,2025-11-17 16:03:02
38934,ARWR.US,2025-11-14,0.998516,5.111676,-0.027346,-0.759709,LATERAL,Compra Fuerte,2025-11-17 16:03:02
319284,MU.US,2025-11-14,0.997033,5.189222,-0.027368,-0.578056,SIN_ESTADO,Compra Fuerte,2025-11-17 16:03:02
94433,CIEN.US,2025-11-14,0.997033,4.931309,-0.089046,-0.455433,ALCISTA,Compra Fuerte,2025-11-17 16:03:02
270532,LITE.US,2025-11-14,0.989614,3.828602,-0.104829,-0.506728,ALCISTA,Compra Fuerte,2025-11-17 16:03:02
272032,LLY.US,2025-11-14,0.985905,4.918967,0.000000,-0.347938,ALCISTA,Compra Fuerte,2025-11-17 16:03:02
278034,LRCX.US,2025-11-14,0.980712,3.655366,-0.110704,-0.475018,SIN_ESTADO,Compra Fuerte,2025-11-17 16:03:02
284784,M.US,2025-11-14,0.977003,3.282542,-0.183565,-0.592805,SIN_ESTADO,Compra Fuerte,2025-11-17 16:03:02
203032,GOOG.US,2025-11-14,0.976261,4.140705,-0.050652,-0.296433,ALCISTA,Compra Fuerte,2025-11-17 16:03:02
203782,GOOGL.US,2025-11-14,0.976261,4.106767,-0.051033,-0.300891,ALCISTA,Compra Fuerte,2025-11-17 16:03:02
